### ASSIGNMENT : CAMERA CALIBRATION

#### A few guidelines:
#### Before going further, read the accompanying document CalibrationAssignment.docx  which describes the assignment and the the things you need to do.
#### Write a separate report answering the questions described in CalibrationAssignment.docx. Upload the report (as a .pdf file) as well as the completed .ipynb file with outputs on Blackboard
Each student should work on the assignment independently.

The code provided can be run on Google Colaboratory -- colab.research.google.com -- with only one modification (see below). 

In [0]:
import numpy as np
import cv2
import scipy.io
import os
from numpy.linalg import norm
from matplotlib import pyplot as plt
from numpy.linalg import det
from numpy.linalg import inv
from scipy.linalg import rq
from numpy.linalg import svd

####This part of the code lets you use files from your Google Drive account
####You need not use this if you are not using Google Colaboratory
####Running code on Google Colab

(a) Make a folder on Google Drive, upload this file and all the data files provided to you into the same folder

(b) The last part of the URL will serve as the "folder_id" in the line below

If you are unsure, you can read further here : https://stackoverflow.com/questions/48376580/google-colab-how-to-read-data-from-my-google-drive

In [0]:
folder_id = 'last-part-of-URL' # Enter the last part of the URL of the Google Drive folder here

!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# choose a local (colab) directory to store the data.
local_download_path = os.path.expanduser('~/data')
try:
  os.makedirs(local_download_path)
except: pass

# 2. Auto-iterate using the query syntax
#    https://developers.google.com/drive/v2/web/search-parameters
file_list = drive.ListFile(
    {'q': '\'' + folder_id + "\' in parents"}).GetList()

for f in file_list:
  # 3. Create & download by id.
  print('title: %s, id: %s' % (f['title'], f['id']))
  fname = os.path.join(local_download_path, f['title'])
  print('downloading to {}'.format(fname))
  f_ = drive.CreateFile({'id': f['id']})
  f_.GetContentFile(fname)
  
os.chdir(local_download_path)

####PART 1: Given corresponding 2d points in the image and 3d coordinates with known extrinsics, estimate the camera intrinsics of the form given in the CalibrationAssignment.docx

In [0]:
# Code for Part 1 starts here

# Step 1: Load the data file pt_corres.mat
data_part1 = scipy.io.loadmat('pt_corres.mat')
cam_pts_3D = data_part1['cam_pts_3D']         # Load the 3d points
pts_2D = data_part1['pts_2D']                 # Load the corresponding 2d points

print pts_2D.shape
print cam_pts_3D.shape

# Step 2: Write your code here to compute the camera intrinsics 


# Code for Part 1 ends here

####PART 2: Given 2d points on the image and corresponding 3d points in the world-coordinate system, estimate both intrinsics and extrinsics. You need to fill in the code for the function calibrate() in calib_DLT.ipynb before running the cell below

In [0]:
def calibrate(x,X):
  '''
  This function computes camera projection matrix from 3D scene points
  and corresponding 2D image points with the Direct Linear Transformation (DLT).
  
  Usage:
  P = calibrate(x, X)
  
  Input:
  x: 2xn image points
  X: 3xn scene points
  
  Output:
  P: 3x4 camera projection matrix
  
  '''
  
  # Your code goes here
  
  # Hint 1: Convert to homogeneous coordinates first
  # Hint 2: np.hstack and np.vstack are useful functions
  
  # Warning: The svd function from Numpy returns U, Sigma and V_transpose (not V, unlike Matlab)
  
  
  
  return P

#######################################################################################################################################################################################################

def P_to_KRt(P):
  '''
  
  This function computes the decomposition of the projection matrix into intrinsic parameters, K, and extrinsic parameters Q (the rotation matrix) and t (the translation vector)
  
  Usage:
  K, Q, t = P_to_KRt(P)
  
  Input: 
  P: 3x4 projection matrix
  
  Outputs:
  K: 3x3 camera intrinsics
  Q: 3x3 rotation matrix (extrinsics)
  t: 3x1 translation vector(extrinsics)
  
  '''
  
  M = P[0:3,0:3]
  
  R, Q = rq(M)
    
  K = R/float(R[2,2])
  
  if K[0,0] < 0:
    K[:,0] = -1*K[:,0]
    Q[0,:] = -1*Q[0,:]
    
  if K[1,1] < 0:
    K[:,1] = -1*K[:,1]
    Q[1,:] = -1*Q[1,:]
  
  if det(Q) < 0:
    print 'Warning: Determinant of the supposed rotation matrix is -1'
  
  P_3_3 = np.dot(K,Q)
  
  P_proper_scale = (P_3_3[0,0]*P)/float(P[0,0])
  
  t = np.dot(inv(K), P_proper_scale[:,3])
  
  return K, Q, t

#######################################################################################################################################################################################################

# Code for Part 2 starts here

# Step 1 : Load the data files rubik_2D_pts.mat and rubik_3d_pts.mat

pts_3d = scipy.io.loadmat('rubik_3D_pts.mat')['pts_3d'] # 3D points in the world-coordinate system with one of the corners of the Rubik's cube as the origin; side of each smaller cube = 1 unit
pts_2d = scipy.io.loadmat('rubik_2D_pts.mat')['pts_2d'] # The corresponding 2D points on the image rubik_cube.jpg

print pts_2d.shape
print pts_3d.shape


# Step 2: Get the camera calibration matrix P
  
P = calibrate(pts_2d, pts_3d)     # You need to fill in the code for this function before you execute this part of the code

print 'P = ', P 

# Step 3: Use the function P_to_KRt (already written for you) to decompose P into intrinsics (K) and extrinsics (R and t)
[K, R, t] = P_to_KRt(P)

print 'K = ', K
print 'R = ', R
print 't = ', t

# Write code here to compute the average reprojection error (averaged over the 28 points given)

# Display the given 2D points and the reprojected 2D points on the Rubik's Cube image provided

# Code for Part 2 ends here